# Microscope images improvement by Cycle GAN

-
-
### Importing Libraries
-
-

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
import tensorflow as tf
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from numpy import vstack
from numpy import asarray
from tensorflow.keras.preprocessing.image import load_img,img_to_array, array_to_img,ImageDataGenerator
from numpy import savez_compressed
from numpy import load

# example of training a cyclegan on the horse2zebra dataset
from random import random
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from keras.models import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

# Importing Image class from PIL module
from PIL import Image

-
-
### Data Preprocessing
-
-

#### A) Separating the bad and good images
The dataset of images contains mix images of bad and good and the labels are mentioned in Excel file for each image whether it is good or bad. I use sql to read and separate the bad images records from the good one and store in separate file.
The below code takes the bad image name from the file and find it in image folder. if it finds the image there, it simply move that image into another folder. this is how it is separating the images.

In [ ]:
# Importing and cleaning dataset
import csv
import pandas as pd
import openpyxl  
import shutil

import os
j=0
k=0

path='D:/Masters/Machine Learning/scopeImage/images/'

df=pd.read_csv('D:/Masters/Machine Learning/scopeImage/Bad_final_csv.csv')
names =df['filenaame'].unique()
for i in names:
    listOfFile = os.listdir(path)
    if i in listOfFile:
        j+=1
        shutil.move(path+i, 'D:/Masters/Machine Learning/scopeImage/TrainA')
    else:
        k+=1
        
        
print("Good files moved =",j)   
print("files not found =",k)  

#### B) Crop images 

As each image boundires are black, showing that some one has edit them in a good way. so i need to remove those black boundries in order to have better data augumentations 


In [ ]:


def image_crop(image):
# Opens a image in RGB mode
    im = image
 
    # Setting the points for cropped image
    left = 10
    top = 10
    right = 768
    bottom = 500
 
    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((left, top, right, bottom))
    return im1


#### C) Loading and Image Augumentation 

Now i will load the images and perform data augmentation on them to increase the size of dataset.

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='wrap')

def load_images_Augumentation(path,path_destination,im_prefix,batch_size, size=(256,256)):
    for filename in listdir(path):
        pixels=load_img(path+filename) #, target_size=size

        img =image_crop(pixels)
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=path_destination, save_prefix=im_prefix, save_format='jpg'):
            i += 1
            if i > batch_size:
                break  # otherwise the generator would loop indefinitely
        


In [ ]:
path='D:/Masters/Machine Learning/scopeImage/'

load_images_Augumentation(path+'bad/',path+'trainBad/','low',10 )
load_images_Augumentation(path+ 'good/', path+'trainGood/', 'high',4)

#### D) Loading images into notebook and then convert to array

Now its time to load all images and convert them into array for futher model building

In [ ]:
def load_images(path, size=(256,256)):
    data_list=list()
    for filename in listdir(path):
        pixels=load_img(path+filename, target_size=size)
        pixels=img_to_array(pixels)
        data_list.append(pixels)
    return asarray(data_list)

In [ ]:
#Data main path
path='D:/Masters/Machine Learning/scopeImage/'
dataA=load_images(path+'trainBad/')

print("The shape of not good  images :", dataA.shape)

dataB=load_images(path+"trainGood/")

print("The Shape of the good images:", dataB.shape)


In [ ]:
filename = 'microscope_for_corena.npz'
savez_compressed(path +filename, dataA, dataB)
print('Saved dataset: ', filename)

Now plotting the Images from npz compressed file

In [ ]:
path='../input/microscope-500-128x128/'
filename = 'microscope_500_128x128.npz'
data= load(path+filename)
dataA,dataB=data['arr_0'],data['arr_1']

In [ ]:
fig = plt.figure(figsize=(15, 15))
fig.add_subplot(3, 3, 1)

plt.imshow(dataA[1].astype('uint8'))
plt.title("Not Good")
fig.add_subplot(3, 3, 2)
plt.imshow(dataA[100].astype('uint8'))
plt.title("Not Good")
fig.add_subplot(3, 3, 3)
plt.imshow(dataA[50].astype('uint8'))
plt.title("Not Good")



In [ ]:
fig = plt.figure(figsize=(15, 15))
fig.add_subplot(1, 3, 1)
plt.imshow(dataB[1].astype('uint8'))
plt.title("Good")

fig.add_subplot(1, 3, 2)
plt.imshow(dataB[100].astype('uint8'))
plt.title("Good")

fig.add_subplot(1, 3, 3)
plt.imshow(dataB[400].astype('uint8'))
plt.title("Good")

### Creating Cycle GAN Model

#### Storing Key point
The CycleGAN discriminator uses InstanceNormalization instead of BatchNormalization.

In [ ]:
# Discriminator

def define_discriminator(image_shape):
    init=RandomNormal(stddev=0.05)
    in_image=Input(shape=image_shape)
    add=Conv2D(64,(4,4),strides=(2,2),padding='same',kernel_initializer=init)(in_image)
    add=LeakyReLU(alpha=0.2)(add)
    
    #layer 2
    add=Conv2D(128, (4,4),strides=(2,2),padding='same',kernel_initializer=init)(add)
    add=InstanceNormalization(axis=-1)(add)
    add=LeakyReLU(alpha=0.2)(add)
    
    add=Conv2D(256, (4,4),strides=(2,2),padding='same',kernel_initializer=init)(add)
    add=InstanceNormalization(axis=-1)(add)
    add=LeakyReLU(alpha=0.2)(add)
    
    add=Conv2D(512, (4,4),strides=(2,2),padding='same',kernel_initializer=init)(add)
    add=InstanceNormalization(axis=-1)(add)
    add=LeakyReLU(alpha=0.2)(add)
    
    add=Conv2D(512, (4,4),strides=(2,2),padding='same',kernel_initializer=init)(add)
    add=InstanceNormalization(axis=-1)(add)
    add=LeakyReLU(alpha=0.2)(add)
    
    patch_output=Conv2D(1, (4,4),padding='same',kernel_initializer=init)(add)
    
    model= Model(in_image,patch_output)
    model.compile(loss='mse', optimizer=Adam(lr=0.002, beta_1=0.5), loss_weights=[0.5])
    return model
    


The generator is an encoder-decoder model architecture. The model takes a source image (e.g. horse photo) and generates a target image (e.g. zebra photo). It does this by first downsampling or encoding the input image down to a bottleneck layer, then interpreting the encoding with a number of ResNet layers that use skip connections, followed by a series of layers that upsample or decode the representation to the size of the output image.

### Resnet Block

In [ ]:
def resnet_block(n_filters, input_layer):
    # weight initialization
    init=RandomNormal(stddev=0.02)
    #First layer convolutional layer
    
    g=Conv2D(n_filters,(3,3),padding='same',kernel_initializer=init)(input_layer)
    g=InstanceNormalization(axis=-1)(g)
    g=Activation('relu')(g)
    
    #Second convolution layer
    g=Conv2D(n_filters, (3,3), padding='same',kernel_initializer=init)(g)
    g=InstanceNormalization(axis=-1)(g)
    
    g=Concatenate()([g,input_layer])
    return g

### Implementing Generator

In [ ]:
def define_generator(image_shape, n_resnet=9):
    
    init=RandomNormal(stddev=0.02)
    #image Input
    in_image=Input(shape=image_shape)
    
    # 64 Filter
    g=Conv2D(64, (7,7), padding='same', kernel_initializer=init )(in_image)
    g=Activation('relu')(g)
    
    # 128 filter
    g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    
    # 256
    g=Conv2D(256,(3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
    g = InstanceNormalization(axis=-1)(g)
    g = Activation('relu')(g)
    
    # Now creating 9 blocks of resnet in the architecture 
    for i in range(n_resnet):
        g=resnet_block(256,g)

    #Now expanding the image from here
    # 128 filter size
    g=Conv2DTranspose(128,(3,3), strides=(2,2), padding='same', kernel_initializer=init )(g)
    g=InstanceNormalization(axis=-1)(g)
    g=Activation('relu')(g)
    
    #64 filter size
    g=Conv2DTranspose(64,(3,3), strides=(2,2), padding='same', kernel_initializer=init )(g)
    g=InstanceNormalization(axis=-1)(g)
    g=Activation('relu')(g)
    
    # size filter 3
    g=Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
    g=InstanceNormalization(axis=-1)(g)
    out_image=Activation('tanh')(g)
    
    #Define Model
    model=Model(in_image,out_image)
    return model
    

In [ ]:
def define_composite_model(g_model_1,d_model,g_model_2,image_shape):
    g_model_1.trainable=True
    d_model.trainable=False
    g_model_2.trainable=False
    

    #discriminator elements 
    input_gen=Input(shape=image_shape)
    gen1_out=g_model_1(input_gen)
    output_d=d_model(gen1_out)
    
    #identity element
    input_id=Input(shape=image_shape)
    output_id=g_model_1(input_id)
    
    #forward
    output_f=g_model_2(gen1_out)
    
    #Backward cycle
    gen2_out=g_model_2(input_id)
    output_b=g_model_1(gen2_out)
    
    
    #define the model complete graph
    model=Model([input_gen,input_id],[output_d,output_id,output_f,output_b])
    #define optimization algoithm configuration
    opt=Adam(lr=0.0002,beta_1=0.5)
    #compile mode
    model.compile(loss=['mse','mae','mae','mae'],loss_weights=[1,5,10,10],  optimizer=opt)
    return model    

In [ ]:

# load and prepare training images
def load_real_samples(filename):
    # load the dataset
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return X, y

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
    # generate fake instance
    X = g_model.predict(dataset)
    # create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [ ]:
def save_models(step, g_model_AtoB, g_model_BtoA):
    # save the first generator model
    filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
    g_model_AtoB.save(filename1)
    # save the second generator model
    filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
    g_model_BtoA.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
    # select a sample of input images
    X_in, _ = generate_real_samples(trainX, n_samples, 0)
    # generate translated images
    X_out, _ = generate_fake_samples(g_model, X_in, 0)
    # scale all pixels from [-1,1] to [0,1]
    X_in = (X_in + 1) / 2.0
    X_out = (X_out + 1) / 2.0
    # plot real images
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_in[i])
    # plot translated image
    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_out[i])
    # save lot to file
    filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
    pyplot.savefig(filename1)
    pyplot.close()

In [ ]:

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
    selected = list()
    for image in images:
        if len(pool) < max_size:
            # stock the pool
            pool.append(image)
            selected.append(image)
        elif random() < 0.5:
            # use image, but don't add it to the pool
            selected.append(image)
        else:
            # replace an existing image and use replaced image
            ix = randint(0, len(pool))
            selected.append(pool[ix])
            pool[ix] = image
    return asarray(selected)

In [ ]:
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
    # define properties of the training run
    n_epochs, n_batch, = 50, 1
    # determine the output square shape of the discriminator
    n_patch = d_model_A.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # prepare image pool for fakes
    poolA, poolB = list(), list()
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
        X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
        # update fakes from pool
        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        # update generator B->A via adversarial and cycle loss
        g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        # update discriminator for A -> [real/fake]
        dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
        dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
        # update generator A->B via adversarial and cycle loss
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        # update discriminator for B -> [real/fake]
        dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
        dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        # summarize performance
        print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            # plot A->B translation
            summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
            # plot B->A translation
            summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
        if (i+1) % (bat_per_epo * 5) == 0:
            # save the models
            save_models(i, g_model_AtoB, g_model_BtoA)

In [ ]:
# load image data
path='../input/microscope-500-128x128/'
filename = 'microscope_500_128x128.npz'
dataset = load_real_samples(path+filename)
print('Loaded')
print("Not Good Images", dataset[0].shape)
print("Good Images", dataset[1].shape)

# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
def save_models(step, g_model_AtoB, g_model_BtoA):
    # save the first generator model
    filename1 = './g_model_AtoB_%06d.h5' % (step+1)
    g_model_AtoB.save(filename1)
    # save the second generator model
    filename2 = './g_model_BtoA_%06d.h5' % (step+1)
    g_model_BtoA.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

Evaluation


In [ ]:
def select_sample(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    return X
# plot the image, the translation, and the reconstruction
def show_plot(imagesX, imagesY1, imagesY2):
    images = vstack((imagesX, imagesY1, imagesY2))
    titles = ['Real', 'Generated', 'Reconstructed']
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    # plot images row by row
    fig = plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        # define subplot
        fig.add_subplot(1, len(images), 1 + i)
        #pyplot.subplot(1, len(images), 1 + i)
        # turn off axis
        
        # plot raw pixel data
        plt.imshow(images[i])
        plt.axis('off')
        # title
        plt.title(titles[i])
    pyplot.show()

In [ ]:
# Loading sample dataset
A_data, B_data = load_real_samples('../input/microscope/microscope_for_corena.npz')
print('Loaded', A_data.shape, B_data.shape)

In [ ]:
from keras.models import load_model
# load the models
cust = {'InstanceNormalization': InstanceNormalization}
model_AtoB = load_model('../input/modelsss/g_model_AtoB_025000.h5', cust)
model_BtoA = load_model('../input/modelsss/g_model_BtoA_025000.h5', cust)

In [ ]:

# plot A->B->A
A_real = select_sample(A_data, 1)
B_generated  = model_AtoB.predict(A_real)
A_reconstructed = model_BtoA.predict(B_generated)
show_plot(A_real, B_generated, A_reconstructed)

In [ ]:
# plot B->A->B
B_real = select_sample(B_data, 1)
A_generated  = model_BtoA.predict(B_real)
B_reconstructed = model_AtoB.predict(A_generated)
show_plot(B_real, A_generated, B_reconstructed)